# Family Lending Fine-Tuning Notebook
This notebook prepares and fine-tunes an LLM on internal family lending dataset examples.

**Sections**:
1. Environment & Dependencies
2. Data Loading & Cleaning
3. Exploratory Stats / Quality Checks
4. Formatting for Fine-Tuning (JSONL)
5. Training / API Fine-Tune Job Submission
6. Evaluation & Error Analysis
7. Save Artifacts / Outputs

> Keep raw source data out of git (already ignored via fine_tuning/raw & datasets/raw).

## 1. Environment & Dependencies
Sets up directories, imports libraries, and confirms the working environment so later steps (data loading, generation, and fine‑tune) run predictably.

In [ ]:
# 1. Environment & Dependencies
import os, json, pandas as pd, pathlib, datetime as dt
from typing import Dict, List
DATA_DIR = pathlib.Path('datasets')
RAW_DIR = DATA_DIR / 'raw'  # git-ignored
PROCESSED_DIR = DATA_DIR / 'processed'
PROCESSED_DIR.mkdir(exist_ok=True, parents=True)
print('Working directory:', os.getcwd())
print('Data dir exists:', DATA_DIR.exists())

## 2. Data Loading & Cleaning
Loads any real seed data (if present) from datasets/raw. Currently a placeholder; safe to skip if only using synthetic generation.

In [ ]:
# 2. Data Loading & Cleaning (placeholder)
# Replace with real load logic. Example expects a CSV in datasets/raw/family_lending_dataset.csv
csv_path = RAW_DIR / 'family_lending_dataset.csv'
if csv_path.exists():
    df = pd.read_csv(csv_path)
    print('Loaded rows:', len(df))
    # Minimal clean example
    df = df.drop_duplicates().reset_index(drop=True)
else:
    print('CSV not found at', csv_path)
    df = pd.DataFrame()
df.head()

## 3. Exploratory Stats / Quality Checks
Basic EDA on any real loaded data. Skips automatically if no dataset is found.

In [ ]:
# 3. Exploratory Stats / Quality Checks (placeholder)
if not df.empty:
    display(df.describe(include='all').transpose())
    print('Null counts:')
    print(df.isnull().sum())
else:
    print('DataFrame empty; skipping EDA')

## 4. Synthetic Chat Dataset Generation
Creates 3,000 structured chat examples (train/val split 80/20) following the approved schema for fine-tuning.

In [ ]:
# 4. Synthetic Dataset Generation (chat-format JSONL) per agreed spec
import random, math
from dataclasses import dataclass

random.seed(42)

TOTAL = 3000
SPLIT_TRAIN = 0.80
# Class counts (fixed)
counts = {
    ('approved', True): 900,    # gift approved
    ('approved', False): 1800,  # loan approved
    ('declined', False): 150,
    ('pending', False): 150,
}
# Sanity check
assert sum(counts.values()) == TOTAL, 'Counts do not sum to TOTAL'

purpose_categories = [
    'medical','education','housing','emergency','small business','travel','debt consolidation',
    'celebrations','vehicle','childcare','groceries','lunch','breakfast','dinner','gas money'
]
relationships = ['mother','father','sister','brother','aunt','uncle','cousin','niece','nephew','grandparent','in-law','close friend']

# Amount ranges & distributions
# Gifts: micro 40%, everyday 40%, special 20%
gift_ranges = [
    ('micro', 10, 40, 0.40),
    ('everyday', 40, 120, 0.40),
    ('special', 120, 250, 0.20),
]
# Non-gift loan buckets with distribution: light 35%, medium 50%, larger 14%, upper 1%
loan_ranges = [
    ('light', 150, 600, 0.35),
    ('medium', 600, 2000, 0.50),
    ('larger', 2000, 6000, 0.14),
    ('upper', 6000, 10000, 0.01),
]

# Interest: include small symbolic interest (1-3%) in <=5% of non-gift loans
INTEREST_RATE_PROB = 0.05

# Helper weighted choice
def weighted_choice(ranges):
    r = random.random()
    cumulative = 0
    for name, lo, hi, w in ranges:
        cumulative += w
        if r <= cumulative:
            return name, random.randint(lo, hi // 10) * 10  # round to nearest $10
    # fallback last
    name, lo, hi, w = ranges[-1]
    return name, random.randint(lo, hi // 10) * 10

empathy_openers = [
    "I appreciate you reaching out and being open about the need.",
    "Thanks for trusting me with this—family support matters.",
    "I hear you and want to help in a way that feels respectful.",
    "You did the right thing by asking early so we can set gentle expectations.",
    "I’m glad you felt comfortable sharing this situation.",
]
positive_reassurance = [
    "Let’s keep this supportive, not stressful.",
    "We’ll structure it so it stays manageable.",
    "I want this to feel like partnership, not pressure.",
    "We’ll keep communication open and flexible.",
]

decline_warmth = [
    "I care about you and this isn’t a rejection of your need.",
    "I wish I could say yes this time—please keep me in the loop.",
    "This is a timing issue on my side, not a lack of support.",
]

pending_warmth = [
    "I just need a little more clarity before finalizing.",
    "Let me review my upcoming commitments and circle back shortly.",
    "I want to help; I just need to confirm a couple details first.",
]

reminder_styles = [
    'gentle monthly check-ins','milestone reminders only','proactive but low-pressure','biweekly friendly note','quarterly recap'
]

def choose_amount(gift: bool, purpose: str):
    if gift:
        # Purpose heuristic: meals/gas -> micro bias
        if purpose in ['lunch','breakfast','dinner','gas money','groceries']:
            # Increase chance of micro
            micro = [r for r in gift_ranges if r[0]=='micro'][0]
            name, lo, hi, w = micro
            return random.randint(lo, hi)  # keep small granular
        name, amt = weighted_choice(gift_ranges)
        return amt
    # Non gift loan
    # Purpose-based overrides
    if purpose in ['lunch','breakfast','dinner','gas money','groceries']:
        # light cap
        return random.randint(150, 300)
    if purpose in ['education','medical','vehicle','housing','small business','debt consolidation']:
        bucket, amt = weighted_choice(loan_ranges)
        return amt
    if purpose == 'celebrations':
        if random.random() < 0.10:  # occasional loan for larger celebration
            return random.randint(250, 600)
        else:
            return random.randint(120, 300)
    if purpose == 'emergency':
        # Bias to medium range
        return random.randint(600, 2000)
    if purpose == 'childcare':
        return random.randint(400, 1200)
    # default
    bucket, amt = weighted_choice(loan_ranges)
    return amt


def build_user_request(purpose: str, amount: int, relationship: str, gift: bool):
    templates = [
        "Hey, could you help me with ${amt} for a {purpose} expense?",
        "I’m hoping you might be able to spot me ${amt} related to {purpose}.",
        "Could you lend me ${amt}? It’s for {purpose} and I’d really appreciate it.",
        "Any chance you can help with ${amt} toward {purpose}?",
        "I could use ${amt} to cover some {purpose} costs—can you help?",
    ]
    base = random.choice(templates).format(amt=amount, purpose=purpose)
    if not gift:
        # Add repayment intent line
        repayment_lines = [
            "I can start repayments next month.",
            "I’d like to pay it back over the next few months.",
            "Happy to set a schedule that works for you.",
            "I can do equal monthly payments if approved.",
        ]
        base += " " + random.choice(repayment_lines)
    else:
        gift_lines = [
            "If this can be a gift I’ll pay it forward.",
            "If you’re okay treating it as a gift, I’m grateful.",
            "Totally understand if not, but hoping this can be a gift.",
        ]
        if random.random() < 0.5:
            base += " " + random.choice(gift_lines)
    return base


def build_assistant_completion(status: str, gift: bool, amount: int, purpose: str, relationship: str):
    currency = 'USD'
    if status == 'approved':
        opener = random.choice(empathy_openers) + ' ' + random.choice(positive_reassurance)
    elif status == 'declined':
        opener = random.choice(decline_warmth)
    else:
        opener = random.choice(pending_warmth)

    lines = [opener]
    lines.append(f"Approval: {status}")
    # Mandatory Terms header
    terms = [f"- Amount: {amount} {currency}"]
    # Disbursement logic
    disburse_options = ['within 2 days (bank transfer)','same day (mobile transfer)','this week (bank transfer)','within 24 hours (instant)']
    terms.append(f"- Disbursement: {random.choice(disburse_options)}")

    if not gift and status == 'approved':
        # Repayment schedule based on amount
        if amount <= 600:
            sched_options = ['3 monthly payments','4 monthly payments','6 biweekly payments']
        elif amount <= 2000:
            sched_options = ['6 monthly payments','9 monthly payments','12 monthly payments']
        else:
            sched_options = ['12 monthly payments','15 monthly payments','18 monthly payments']
        terms.append(f"- Repayment Schedule: {random.choice(sched_options)} starting next month")
        # Interest occasionally
        if random.random() < INTEREST_RATE_PROB:
            terms.append(f"- Interest: {random.randint(1,3)}% symbolic family rate")
        else:
            terms.append("- Interest: 0% (family support)")
        # Grace
        if amount <= 600:
            if random.random() < 0.4:
                terms.append("- Grace: 2 weeks flexible")
        elif amount <= 2000:
            if random.random() < 0.6:
                terms.append("- Grace: 1 month flexible")
        else:
            terms.append("- Grace: 1 month flexible")
        # Reminder Style
        terms.append(f"- Reminder Style: {random.choice(reminder_styles)}")

    notes = []
    if gift:
        notes.append("Treated as a gift—no repayment expected.")
    if status == 'pending':
        notes.append("Need confirmation of exact timing and any other contributions.")
    if status == 'declined':
        notes.append("Please reach out again if circumstances change; I want to help where I can.")

    lines.append('Terms:')
    lines.extend(terms)
    if notes:
        lines.append('Notes: ' + ' '.join(notes))

    return "\n".join(lines)

# Build label list
labels = []
for (status, gift), c in counts.items():
    labels.extend([(status, gift)] * c)
random.shuffle(labels)

examples = []
for i, (status, gift) in enumerate(labels):
    purpose = random.choice(purpose_categories)
    relationship = random.choice(relationships)
    amount = choose_amount(gift, purpose)
    user_req = build_user_request(purpose, amount, relationship, gift)
    prompt_context = f"User Request:\n{user_req}\nContext:\nRelationship: {relationship}\nPurpose: {purpose}\nGift Classification: {'yes' if gift else 'no'}"
    completion = build_assistant_completion(status, gift, amount, purpose, relationship)
    record = {
        'messages': [
            {'role': 'user', 'content': prompt_context},
            {'role': 'assistant', 'content': completion}
        ]
    }
    examples.append(record)

# Split
train_size = int(TOTAL * SPLIT_TRAIN)
train_set = examples[:train_size]
val_set = examples[train_size:]

FT_DIR = pathlib.Path('fine_tuning')
FT_DIR.mkdir(exist_ok=True, parents=True)
train_path = FT_DIR / 'family_lending_chat_train.jsonl'
val_path = FT_DIR / 'family_lending_chat_val.jsonl'

with open(train_path, 'w', encoding='utf-8') as f:
    for rec in train_set:
        f.write(json.dumps(rec, ensure_ascii=False) + '\n')
with open(val_path, 'w', encoding='utf-8') as f:
    for rec in val_set:
        f.write(json.dumps(rec, ensure_ascii=False) + '\n')

print('Wrote', len(train_set), 'train and', len(val_set), 'validation examples.')

# Quick distribution check
from collections import Counter
ctr = Counter((rec['messages'][1]['content'].split('\n')[1].split(': ')[1], 'yes' if 'Gift Classification: yes' in rec['messages'][0]['content'] else 'no') for rec in examples)
print('Class distribution (status,gift):')
for k,v in ctr.items():
    print(k, v)

examples[0]  # show one sample record

In [ ]:
# 5. (Placeholder) Fine-Tune Process Outline for Colab + Ollama style adapter
# This cell will later be expanded to actually launch a QLoRA / LoRA style fine-tune using an 8B base (e.g., llama3:8b) if feasible.
# High-level steps (will add executable code after confirming environment in Colab):
# 1. !pip install transformers peft datasets accelerate bitsandbytes sentencepiece
# 2. Load base model in 4-bit (bnb.int4) with transformers.
# 3. Tokenize chat examples -> apply a chat template or manual join (system(optional) + user + assistant).
# 4. Apply LoRA config (rank ~ 32, alpha 16, dropout 0.05) with target modules depending on architecture.
# 5. Train with gradient checkpointing, batch size adaptation, and eval on validation set.
# 6. Save adapter weights; optionally merge for export.
# 7. Convert to GGUF / create Ollama Modelfile referencing the adapter or merged model.
# 8. Test sample generations locally via ollama run.

print('Fine-tune outline placeholder ready. Expand when running in Colab environment.')

## 5. Fine-Tune Outline (To Implement in Colab)
Guides setting up a QLoRA/LoRA run on a base model (e.g., llama3:8b) using 4-bit loading for free-tier GPU constraints.

In [ ]:
# 6. (Placeholder) Evaluation & Sample Generation
# After fine-tuning, load adapter/merged model and run sample prompts to qualitatively assess.
# - Check gift vs non-gift formatting consistency
# - Ensure optional fields omitted for gifts
# - Spot check declined/pending tone warmth
# - Possibly compute simple regex-based metric: required headers present.

print('Add evaluation logic after training step implemented.')

In [ ]:
# 7. Save Artifacts / Outputs (placeholder)
# When training is complete, implement logic here to:
# - Save adapter weights (e.g., ./fine_tuning/adapter)
# - Optionally merge LoRA into base model (careful with Colab RAM)
# - Export a Modelfile snippet for Ollama referencing merged or adapter path
# - Write summary stats (counts, example) to a JSON report

print('Artifact saving placeholder ready.')

## 6. Evaluation & Sample Generation
Will load the fine-tuned adapter/merged model to run qualitative checks and simple structural validation metrics.

## 7. Save Artifacts / Outputs
Persist key outputs (trained adapter, merged model if desired, dataset stats, sample generations) and prepare an Ollama Modelfile snippet for local deployment.